## データベースとスキーマを作成

In [0]:
CREATE DATABASE IF NOT EXISTS t_kamei_db;

In [0]:
CREATE SCHEMA IF NOT EXISTS t_kamei_db.cddn_demo_schema;

## デフォルトのデータベースとスキーマを変更

In [0]:
USE DATABASE t_kamei_db;
USE SCHEMA cddn_demo_schema;

## Snowflake Semantic Views を定義

In [0]:
-- LINEITEM の Semantic View を定義
CREATE OR REPLACE SEMANTIC VIEW TPCH_LINEITEM_SV
  TABLES (
    line_items AS SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.LINEITEM
      PRIMARY KEY (l_orderkey, l_linenumber),
    orders AS SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS
      PRIMARY KEY (o_orderkey),
    customer AS SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER
      PRIMARY KEY (c_custkey),
    nation   AS SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.NATION
      PRIMARY KEY (n_nationkey)
  )

  RELATIONSHIPS (
    li_to_orders AS line_items(l_orderkey) REFERENCES orders(o_orderkey),
    orders_to_cust AS orders(o_custkey)        REFERENCES customer(c_custkey),
    cust_to_nation AS customer(c_nationkey)    REFERENCES nation(n_nationkey)
  )

  FACTS (
    line_items.gross_revenue   AS l_extendedprice * (1 - l_discount),
    line_items.is_returned     AS IFF(l_returnflag = 'R', 1, 0),

    -- 90日以内に出荷された行なら注文キーを返す
    line_items.order_key_if_shipped_90d AS
      IFF(DATEDIFF('day', orders.order_date, line_items.ship_date) BETWEEN 0 AND 90,
          l_orderkey, NULL)
  )

  DIMENSIONS (
    orders.order_date      AS CAST(o_orderdate AS DATE),
    orders.order_month     AS DATE_TRUNC('month', o_orderdate),  
    line_items.ship_date     AS CAST(l_shipdate AS DATE),
    line_items.ship_month    AS DATE_TRUNC('month', l_shipdate),

    -- 追加（customer / nation）
    customer.customer_name AS c_name,
    nation.nation_key     AS n_nationkey,
    nation.nation_name    AS n_name
 )

  METRICS (
    ----- 1. SIMPLE（総収益） -----
    line_items.total_revenue  AS SUM(line_items.gross_revenue),

    ----- 2. RATIO（返品率） -----
    line_items.total_lines_metric  AS COUNT(*),
    line_items.returned_lines_metric AS SUM(line_items.is_returned),
    line_items.return_rate AS returned_lines_metric / NULLIF(total_lines_metric, 0),

    ----- 3. DERIVED（前日の総収益） -----
    line_items.total_revenue_prev_day AS
      LAG(line_items.total_revenue) OVER (
        PARTITION BY EXCLUDING line_items.ship_date
        ORDER BY line_items.ship_date
      ),

    ----- 4. CUMULATIVE（月内累計総収益） -----
    line_items.revenue_mtd AS
      SUM(line_items.total_revenue) OVER (
        PARTITION BY EXCLUDING line_items.ship_date
        ORDER BY line_items.ship_date
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
      ),

    ----- 5. CONVERSION（注文→出荷90日以内達成率） -----
    -- 90日内に「1回でも」出荷のあった注文数（サブクエリ禁止の回避：条件付き DISTINCT）
    line_items.orders_shipped_90d AS COUNT(DISTINCT line_items.order_key_if_shipped_90d),

    -- 発注数（重複排除で注文粒度に）
    line_items.orders_placed AS COUNT(DISTINCT line_items.l_orderkey),

    -- コンバージョン率（派生：メトリクス同士の式）
    order_to_ship_conversion_rate_90d AS
      line_items.orders_shipped_90d / NULLIF(line_items.orders_placed, 0)
  );

In [ ]:
-- ディメンション一覧
SHOW SEMANTIC DIMENSIONS IN t_kamei_db.cddn_demo_schema.TPCH_LINEITEM_SV;

In [ ]:
-- メトリック一覧
SHOW SEMANTIC METRICS IN t_kamei_db.cddn_demo_schema.TPCH_LINEITEM_SV;

## Snowflake Semantic Views からデータを取得

### 1. SIMPLE（総収益）

In [0]:
-- 自然言語クエリ例： 1993年1月の国ごとの総収益を教えてください
SELECT *
FROM SEMANTIC_VIEW(
  TPCH_LINEITEM_SV
  DIMENSIONS
    nation.nation_name AS nation_name
  METRICS
    line_items.total_revenue AS total_revenue
  WHERE 
    line_items.ship_date >= DATE '1993-01-01'
    AND line_items.ship_date <  DATE '1993-02-01'
)
ORDER BY nation_name;

### 2. RATIO（返品率）

In [0]:
-- 自然言語クエリ例： 1993年1月の日別返品率を教えてください
SELECT *
FROM SEMANTIC_VIEW(
  TPCH_LINEITEM_SV
  DIMENSIONS
    line_items.ship_date AS ship_date
  METRICS
    return_rate                AS return_rate,
    returned_lines_metric      AS returned_lines,
    total_lines_metric         AS total_lines
  WHERE
    line_items.ship_date >= DATE '1993-01-01'
    AND line_items.ship_date <  DATE '1993-02-01'
)
ORDER BY ship_date;

### 3. DERIVED（前日の総収益）

In [0]:
-- 自然言語クエリ例： 1993年2月の日別収益と前日収益を教えてください
-- ＜課題＞このまま 2/1 がデータがなく値をとれない
SELECT *
FROM SEMANTIC_VIEW(
  TPCH_LINEITEM_SV
  DIMENSIONS
    line_items.ship_date AS ship_date
  METRICS
    line_items.total_revenue           AS total_revenue,
    line_items.total_revenue_prev_day  AS total_revenue_prev_day
  WHERE
    line_items.ship_date >= DATE '1993-02-01'
    AND line_items.ship_date <  DATE '1993-03-01'
)
ORDER BY ship_date;

In [0]:
-- ＜暫定対応＞ 前月の取得の日付を調整
WITH rev AS (
  /* 当日分（2/1〜2/28） */
  SELECT
    ship_date,
    total_revenue
  FROM SEMANTIC_VIEW(
    TPCH_LINEITEM_SV
    DIMENSIONS
      line_items.ship_date AS ship_date
    METRICS
      line_items.total_revenue AS total_revenue
    WHERE
      line_items.ship_date >= DATE '1993-02-01'
      AND line_items.ship_date <  DATE '1993-03-01'
  )
),
rev_prev AS (
  /* 前日分を取得するため、1/31〜2/28 を取り、+1日シフトして当日に合わせる */
  SELECT
    DATEADD(day, 1, ship_date) AS ship_date,
    total_revenue               AS total_revenue_prev_day
  FROM SEMANTIC_VIEW(
    TPCH_LINEITEM_SV
    DIMENSIONS
      line_items.ship_date AS ship_date
    METRICS
      line_items.total_revenue AS total_revenue
    WHERE
      line_items.ship_date >= DATE '1993-01-31'  -- 前日参照用に 1/31 を含める
      AND line_items.ship_date <  DATE '1993-03-01'
  )
)
SELECT
  r.ship_date,
  r.total_revenue,
  p.total_revenue_prev_day
  -- 必要なら NULL を 0 で埋める: COALESCE(p.total_revenue_prev_day, 0) AS total_revenue_prev_day
FROM rev r
LEFT JOIN rev_prev p USING (ship_date)
ORDER BY r.ship_date;

In [ ]:
-- 国ごとに取得
SELECT *
FROM SEMANTIC_VIEW(
  TPCH_LINEITEM_SV
  DIMENSIONS
    nation.nation_name AS nation_name,
    line_items.ship_date AS ship_date
  METRICS
    line_items.total_revenue           AS total_revenue,
    line_items.total_revenue_prev_day  AS total_revenue_prev_day
  WHERE
    line_items.ship_date >= DATE '1993-02-01'
    AND line_items.ship_date <  DATE '1993-03-01'
)
ORDER BY nation_name, ship_date;

### 4. CUMULATIVE（月内累計総収益）

In [0]:
-- 1993年2月の日別累計収益を教えてください
SELECT *
FROM SEMANTIC_VIEW(
  TPCH_LINEITEM_SV
  DIMENSIONS
    line_items.ship_date  AS ship_date,
    line_items.ship_month AS ship_month
  METRICS
    line_items.total_revenue AS total_revenue,
    line_items.revenue_mtd    AS revenue_mtd
  WHERE
    line_items.ship_date >= DATE '1993-02-01'
    AND line_items.ship_date <  DATE '1993-03-01'
)
ORDER BY ship_date;

### 5. CONVERSION（注文→出荷90日以内達成率）

In [0]:
-- 1993年2月1日のコンバージョン率を教えてください
-- 再現できなかったため、verified_query で指定
SELECT *
FROM SEMANTIC_VIEW(
  TPCH_LINEITEM_SV
  DIMENSIONS
    orders.order_month AS order_month
  METRICS
    order_to_ship_conversion_rate_90d AS conv_rate_90d
  WHERE
    orders.order_month = DATE '1993-02-01'
)
ORDER BY order_month;

In [0]:
-- あっているか確認
WITH od AS (
    SELECT
        o_orderkey,
        o_orderdate
    FROM
        SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS
    WHERE
        o_orderdate >= DATE '1993-02-01'
        AND o_orderdate < DATE '1993-03-01'
),
converted AS (
    SELECT
        DISTINCT od.o_orderkey
    FROM
        od
        JOIN SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.LINEITEM li ON li.l_orderkey = od.o_orderkey
    WHERE
        li.l_shipdate >= od.o_orderdate
        AND li.l_shipdate < DATEADD(day, 91, od.o_orderdate)
)
SELECT
    COUNT(*) AS base_orders,
    (
        SELECT
            COUNT(*)
        FROM
            converted
    ) AS converted_orders,
    (
        SELECT
            COUNT(*)
        FROM
            converted
    ) / COUNT(*) AS conversion_rate_pct
FROM
    od;

In [0]:
# end